In [1]:
import io
import random
from tqdm import tqdm, trange
import torch
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from torch.utils.data import  RandomSampler, DataLoader
from model.model_builder_out_model import ModelBuilder
from model.img_resnet_2_2 import ResNet34
from el_dataset2 import  Bert_Res_Data
from torch import optim
from torch.nn import CrossEntropyLoss
import numpy as np

In [2]:
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)
class Option:
    train_file = './data/train'
    val_file = './data/valid'
    test_file = './data/test'
    vid_pic_path = './data/vid'
    cid_pic_path = './data/cid'
    
    bert_model = 'bert-base-chinese'
    res_input_dim = 4096
    res_hidden_dim = 512
    fc_input_dim = 768+768+2048*2
    fc_hidden_dim = 512
    input_size = 227
    class_num = 2
    max_seq_length = 128
    train_batch_size = 24
    eval_batch_size = 16
    num_train_epochs = 10
    do_lower_case = False

opt = Option()

In [3]:
text_model = BertModel.from_pretrained(opt.bert_model)
img_model = ResNet34()
model = ModelBuilder(text_model,img_model,opt)
model.cuda()

02/06/2020 11:52:17 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese.tar.gz from cache at /root/.pytorch_pretrained_bert/42d4a64dda3243ffeca7ec268d5544122e67d9d06b971608796b483925716512.02ac7d664cff08d793eb00d6aac1d04368a1322435e5fe0a27c70b0b3a85327f
02/06/2020 11:52:17 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/42d4a64dda3243ffeca7ec268d5544122e67d9d06b971608796b483925716512.02ac7d664cff08d793eb00d6aac1d04368a1322435e5fe0a27c70b0b3a85327f to temp dir /tmp/tmpixvc9pf7
02/06/2020 11:52:21 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "max_seq_length": 128,
  "num_attention_heads": 12,
  "num_hidd

ModelBuilder(
  (text_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
    

In [4]:
train_data = Bert_Res_Data(opt,tp="train")
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=opt.train_batch_size)

val_data = Bert_Res_Data(opt,tp="valid")
val_sampler = RandomSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=opt.eval_batch_size)

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
02/06/2020 11:22:04 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.pytorch_pretrained_bert/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
02/06/2020 11:22:47 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.pytorch_pretrained_bert/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


In [6]:
ignored_params = list(map(id,model.fc.parameters()))+list(map(id,model.fc1.parameters()))+list(map(id,model.fc2.parameters()))+list(map(id,model.fc3.parameters()))+list(map(id,model.fc4.parameters()))+list(map(id,model.fc5.parameters()))
base_params = filter(lambda p: id(p) not in ignored_params,model.parameters())
optimizer = optim.SGD([
             {'params': base_params},
             {'params': model.fc.parameters(), 'lr': 3e-3},{'params': model.fc1.parameters(), 'lr': 3e-3},
             {'params': model.fc2.parameters(), 'lr': 3e-3},{'params': model.fc3.parameters(), 'lr': 3e-3},
             {'params': model.fc4.parameters(), 'lr': 3e-3},{'params': model.fc5.parameters(), 'lr': 3e-3},
 
             ], lr=3e-4,momentum = 0.9)

In [7]:
out_txt = open("./record/our_model1", "w+", encoding="utf8")
loss_fct = CrossEntropyLoss()
model.train()
acc = 0
for _ in trange(int(5), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    eval_accuracy = 0
    nb_eval_examples = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.cuda() for t in batch)
        vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,label_ids,img1,img2 = batch
        out1,out2,out3,out4,outputs = model(vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,img1,img2)
        
        loss1 = loss_fct(out1.view(-1,opt.class_num), label_ids.view(-1))
        loss2 = loss_fct(out2.view(-1,opt.class_num), label_ids.view(-1))
        loss3 = loss_fct(out3.view(-1,opt.class_num), label_ids.view(-1))
        loss4 = loss_fct(out4.view(-1,opt.class_num), label_ids.view(-1))
        loss5 = loss_fct(outputs.view(-1,opt.class_num), label_ids.view(-1))
        loss = loss1 + loss2 + loss3 + loss4 +loss5
        loss.backward()
        optimizer.step()
        model.zero_grad()


        tr_loss += loss.item()
        nb_tr_examples += vid_input_ids.size(0)
        nb_tr_steps += 1

        outputs = outputs.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        tmp_eval_accuracy = accuracy(outputs, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_examples += vid_input_ids.size(0)


        if (step + 1) % 100 == 0:
            eval_accuracy = eval_accuracy / nb_eval_examples
            
            out_str = "now loss : {} now acc : {}".format(tr_loss/100,eval_accuracy)
            out_txt.write("%s\n" %out_str)
            print ("now loss : {} now acc : {}".format(tr_loss/100,eval_accuracy))
            
            eval_accuracy = 0
            nb_eval_examples = 0
            tr_loss = 0

    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for step, batch in enumerate(val_dataloader):
        batch = tuple(t.cuda() for t in batch)
        vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,label_ids,img1,img2 = batch
        with torch.no_grad():
            out1,out2,out3,out4,logits = model(vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,img1,img2)
            #logits = model(input_ids, segment_ids, input_mask)

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        tmp_eval_accuracy = accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy

        nb_eval_examples += vid_input_ids.size(0)
        nb_eval_steps += 1


    eval_accuracy = eval_accuracy / nb_eval_examples
    
    out_str = "now valid acc : {}".format(eval_accuracy)
    out_txt.write("%s\n" %out_str)
    print (eval_accuracy)
    
    if eval_accuracy >acc:
        acc = eval_accuracy
        torch.save(model.state_dict(),'our_model1.pt')
    else:
        if _ >= 5:
            break

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


now loss : 3.0547576260566713 now acc : 0.684047619047619
now loss : 2.8324169659614564 now acc : 0.7480952380952381
now loss : 2.7129530024528505 now acc : 0.7745238095238095


/opt/conda/lib/python3.6/site-packages/PIL/Image.py:2575: DecompressionBombWarning: Image size (104982930 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


now loss : 2.7155805039405823 now acc : 0.7823809523809524
now loss : 2.5858731198310854 now acc : 0.8042857142857143
now loss : 2.50547204375267 now acc : 0.8202380952380952
now loss : 2.4621807670593263 now acc : 0.825
now loss : 2.4566389656066896 now acc : 0.8328571428571429
now loss : 2.364799156188965 now acc : 0.8426190476190476
now loss : 2.346206728219986 now acc : 0.8473809523809523
now loss : 2.250629140138626 now acc : 0.8580952380952381
0.8725


Epoch:  20%|██        | 1/5 [58:49<3:55:19, 3529.81s/it]

...

Epoch:  60%|██████    | 3/5 [2:56:55<1:57:52, 3536.48s/it]

now loss : 1.2545413410663604 now acc : 0.9783333333333334
now loss : 1.1965620666742325 now acc : 0.9802380952380952
now loss : 1.2085674601793288 now acc : 0.9797619047619047
now loss : 1.2199348360300064 now acc : 0.9778571428571429
now loss : 1.2622296798229218 now acc : 0.9783333333333334
now loss : 1.2703646463155747 now acc : 0.9764285714285714
now loss : 1.2340271282196045 now acc : 0.9747619047619047
now loss : 1.2564669281244278 now acc : 0.9742857142857143
now loss : 1.2764370548725128 now acc : 0.9721428571428572
now loss : 1.2545706486701966 now acc : 0.974047619047619
now loss : 1.3041720026731491 now acc : 0.97


Epoch:  80%|████████  | 4/5 [3:56:06<59:00, 3540.95s/it]  

0.9141
now loss : 1.0497168171405793 now acc : 0.9876190476190476
now loss : 1.0763529980182647 now acc : 0.9852380952380952
now loss : 1.1052417212724686 now acc : 0.9830952380952381
now loss : 1.1125826114416122 now acc : 0.9823809523809524
now loss : 1.1250599616765975 now acc : 0.9847619047619047
now loss : 1.1712414747476578 now acc : 0.9783333333333334
now loss : 1.1507826375961303 now acc : 0.9809523809523809
now loss : 1.1389342993497849 now acc : 0.9840476190476191
now loss : 1.150685847401619 now acc : 0.9792857142857143
now loss : 1.1671765476465226 now acc : 0.9797619047619047
now loss : 1.1017323327064514 now acc : 0.981904761904762
0.9191


Epoch: 100%|██████████| 5/5 [4:55:17<00:00, 3543.81s/it]


In [8]:
ignored_params = list(map(id,model.fc.parameters()))+list(map(id,model.fc1.parameters()))+list(map(id,model.fc2.parameters()))+list(map(id,model.fc3.parameters()))+list(map(id,model.fc4.parameters()))+list(map(id,model.fc5.parameters()))
base_params = filter(lambda p: id(p) not in ignored_params,model.parameters())
optimizer = optim.SGD([
             {'params': base_params},
             {'params': model.fc.parameters(), 'lr': 1e-3},{'params': model.fc1.parameters(), 'lr': 1e-3},
             {'params': model.fc2.parameters(), 'lr': 1e-3},{'params': model.fc3.parameters(), 'lr': 1e-3},
             {'params': model.fc4.parameters(), 'lr': 1e-3},{'params': model.fc5.parameters(), 'lr': 1e-3},

             ], lr=2e-4,momentum = 0.9)

In [9]:
out_txt = open("./record/our_model1", "a+", encoding="utf8")
loss_fct = CrossEntropyLoss()
model.train()
acc = 0
for _ in trange(int(opt.num_train_epochs), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    eval_accuracy = 0
    nb_eval_examples = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.cuda() for t in batch)
        vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,label_ids,img1,img2 = batch
        out1,out2,out3,out4,outputs = model(vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,img1,img2)
        
        loss1 = loss_fct(out1.view(-1,opt.class_num), label_ids.view(-1))
        loss2 = loss_fct(out2.view(-1,opt.class_num), label_ids.view(-1))
        loss3 = loss_fct(out3.view(-1,opt.class_num), label_ids.view(-1))
        loss4 = loss_fct(out4.view(-1,opt.class_num), label_ids.view(-1))
        loss5 = loss_fct(outputs.view(-1,opt.class_num), label_ids.view(-1))
        loss = loss1 + loss2 + loss3 + loss4 +loss5
        loss.backward()
        optimizer.step()
        model.zero_grad()


        tr_loss += loss.item()
        nb_tr_examples += vid_input_ids.size(0)
        nb_tr_steps += 1

        outputs = outputs.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        tmp_eval_accuracy = accuracy(outputs, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_examples += vid_input_ids.size(0)


        if (step + 1) % 100 == 0:
            eval_accuracy = eval_accuracy / nb_eval_examples
            
            out_str = "now loss : {} now acc : {}".format(tr_loss/100,eval_accuracy)
            out_txt.write("%s\n" %out_str)
            print ("now loss : {} now acc : {}".format(tr_loss/100,eval_accuracy))
            
            eval_accuracy = 0
            nb_eval_examples = 0
            tr_loss = 0

    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for step, batch in enumerate(val_dataloader):
        batch = tuple(t.cuda() for t in batch)
        vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,label_ids,img1,img2 = batch
        with torch.no_grad():
            out1,out2,out3,out4,logits = model(vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,img1,img2)
            #logits = model(input_ids, segment_ids, input_mask)

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        tmp_eval_accuracy = accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy

        nb_eval_examples += vid_input_ids.size(0)
        nb_eval_steps += 1


    eval_accuracy = eval_accuracy / nb_eval_examples
    
    out_str = "now valid acc : {}".format(eval_accuracy)
    out_txt.write("%s\n" %out_str)
    print (eval_accuracy)
    
    if eval_accuracy >acc:
        acc = eval_accuracy
        torch.save(model.state_dict(),'our_model1.pt')
    else:
        break

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/PIL/Image.py:2575: DecompressionBombWarning: Image size (104982930 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


now loss : 1.0166408514976502 now acc : 0.9871428571428571


/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


now loss : 0.9409494560956955 now acc : 0.99
now loss : 0.9427064245939255 now acc : 0.9907142857142858
now loss : 0.944284838438034 now acc : 0.9897619047619047
now loss : 0.9668938797712326 now acc : 0.9888095238095238
now loss : 0.9356838083267212 now acc : 0.9907142857142858
now loss : 0.9869342756271362 now acc : 0.9888095238095238
now loss : 0.9637721621990204 now acc : 0.9871428571428571
now loss : 0.9672341680526734 now acc : 0.9895238095238095
now loss : 0.988777517080307 now acc : 0.9885714285714285
now loss : 0.9618419098854065 now acc : 0.9897619047619047
0.9212


Epoch:  10%|█         | 1/10 [59:08<8:52:17, 3548.58s/it]

now loss : 0.8718844294548035 now acc : 0.9923809523809524
now loss : 0.8647763010859489 now acc : 0.9942857142857143
now loss : 0.8683710062503814 now acc : 0.9921428571428571
now loss : 0.9039366239309311 now acc : 0.9916666666666667
now loss : 0.913753269314766 now acc : 0.9907142857142858
now loss : 0.9080692803859711 now acc : 0.991904761904762
now loss : 0.8908811753988266 now acc : 0.9909523809523809
now loss : 0.9109798747301102 now acc : 0.991904761904762
now loss : 0.9027325963973999 now acc : 0.9921428571428571
now loss : 0.9094808495044708 now acc : 0.9916666666666667
now loss : 0.9155768221616745 now acc : 0.9907142857142858
0.9225


Epoch:  20%|██        | 2/10 [1:58:19<7:53:13, 3549.24s/it]

now loss : 0.8489521640539169 now acc : 0.9935714285714285
now loss : 0.8411858195066452 now acc : 0.9938095238095238
now loss : 0.8489653521776199 now acc : 0.9947619047619047
now loss : 0.8461575162410736 now acc : 0.9942857142857143
now loss : 0.8344067996740341 now acc : 0.9938095238095238
now loss : 0.8281187915802002 now acc : 0.9952380952380953
now loss : 0.8655936473608017 now acc : 0.9935714285714285
now loss : 0.8881096476316452 now acc : 0.9930952380952381
now loss : 0.8686260879039764 now acc : 0.9914285714285714
now loss : 0.8394322943687439 now acc : 0.9926190476190476
now loss : 0.862798449397087 now acc : 0.995
0.9213


In [4]:
model.load_state_dict(torch.load('./our_model.pt'))

In [5]:
seed= 1
torch.manual_seed(seed)            # 为CPU设置随机种子
torch.cuda.manual_seed(seed)       # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed) 
random.seed(seed)
np.random.seed(seed)

In [6]:
test_data = Bert_Res_Data(opt,tp="test")
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data,num_workers = 0,shuffle=False,sampler=None,batch_sampler=None,drop_last = False,  batch_size=opt.train_batch_size)

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
02/06/2020 11:52:31 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.pytorch_pretrained_bert/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


In [13]:
from sklearn import metrics
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_test = np.array([])
y_pred = np.array([])
for step, batch in enumerate(test_dataloader):
    vid_guids,vid_input_ids,vid_segment_ids,vid_input_mask,cid_guids,cid_input_ids,cid_segment_ids,cid_input_mask,label_ids,img1,img2 = batch
    batch = [vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,label_ids,img1,img2]
    batch = tuple(t.cuda() for t in batch)
    vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,label_ids,img1,img2 = batch
    
    with torch.no_grad():
        out1,out2,out3,out4,logits = model(vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,img1,img2)
        #logits = model(input_ids, segment_ids, input_mask)
    y_pred = np.concatenate((y_pred, logits.max(1)[1].cpu().numpy()))
    logits = logits.detach().cpu().numpy()
    label_ids = label_ids.to('cpu').numpy()
    y_test = np.concatenate((y_test, label_ids))
    tmp_eval_accuracy = accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += vid_input_ids.size(0)
    nb_eval_steps += 1


eval_accuracy = eval_accuracy / nb_eval_examples
print (eval_accuracy)


/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


0.9281


In [14]:
print(metrics.classification_report(y_test, y_pred, digits=4, target_names=['0','1']))

              precision    recall  f1-score   support

           0     0.9379    0.9442    0.9410      6074
           1     0.9127    0.9032    0.9080      3926

    accuracy                         0.9281     10000
   macro avg     0.9253    0.9237    0.9245     10000
weighted avg     0.9280    0.9281    0.9280     10000



In [7]:
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
out = open("./result_our_model", "w+", encoding="utf8")
for step, batch in enumerate(test_dataloader):
    vid_guids,vid_input_ids,vid_segment_ids,vid_input_mask,cid_guids,cid_input_ids,cid_segment_ids,cid_input_mask,label_ids,img1,img2 = batch
    batch = [vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,label_ids,img1,img2]
    batch = tuple(t.cuda() for t in batch)
    vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,label_ids,img1,img2 = batch
    
    with torch.no_grad():
        out1,out2,out3,out4,logits = model(vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,img1,img2)
        #logits = model(input_ids, segment_ids, input_mask)
    logits = logits.detach().cpu().numpy()
    label_ids = label_ids.to('cpu').numpy()
    
    outputs = np.argmax(logits, axis=1)
    #outputs = (outputs == label_ids)
   # print(vid_guids)
   # print(cid_guids)
   # print(outputs)
    len_num = len(outputs)
    for num in range(len_num):
        line = [vid_guids[num],cid_guids[num],str(outputs[num])]
        out_str = '\t'.join(line)
        out.write("%s\n" %out_str)
    
    tmp_eval_accuracy = accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += vid_input_ids.size(0)
    nb_eval_steps += 1


eval_accuracy = eval_accuracy / nb_eval_examples
print (eval_accuracy)
out.close()

/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


0.9281


In [10]:
print(nb_eval_examples*eval_accuracy)

471.0


In [10]:
result_dict = {}
with open("./result", "r", encoding="utf8") as f:
    lines = f.readlines()
    for i,line in enumerate(lines):
        line = line.strip().split('\t')
        result_dict[line[0]] = line[1]
#print(result_dict)   
total_num = 0
true_num = 0
with open("./data/testl", "r", encoding="utf8") as f:
    lines = f.readlines()
    for i,line in enumerate(lines):
        total_num += 1
        line = line.strip().split('\t')
        if  result_dict[line[1]+line[3]] == 'True':
            true_num += 1
print(true_num/total_num)   

0.9055793991416309


In [7]:
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for step, batch in enumerate(test_dataloader):
    batch = tuple(t.cuda() for t in batch)
    vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,label_ids,img1,img2 = batch
    with torch.no_grad():
        out1,out2,out3,out4,logits = model(vid_input_ids,vid_segment_ids,vid_input_mask,cid_input_ids,cid_segment_ids,cid_input_mask,img1,img2)
        #logits = model(input_ids, segment_ids, input_mask)
    
    logits = logits.detach().cpu().numpy()
    label_ids = label_ids.to('cpu').numpy()
    tmp_eval_accuracy = accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += vid_input_ids.size(0)
    nb_eval_steps += 1


eval_accuracy = eval_accuracy / nb_eval_examples
print (eval_accuracy)

0.9098712446351931
